# Simple Substitution
`w266 Final Project: Crosslingual Word Embeddings`

The code in this notebook was used to develop an algorithm to generate crosslingual word embeddings by training on a monolingual corpus and substituting translations at runtime.

# Notebook Setup

In [5]:
# general imports
from __future__ import print_function
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# tell matplotlib not to open a new window
%matplotlib inline

# autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Set base paths depending on your machine.

In [6]:
## Maya's paths
BASE = '/home/mmillervedam/' #'/Users/mmillervedam/Documents/MIDS/w266' #
PROJ = '/home/mmillervedam/ProjectRepo' #'/Users/mmillervedam/Documents/MIDS/w266/FinalProject'#

## Roseanna's paths
BASE = '/home/rhopper/'
PROJ = '/home/rhopper/W266-Fall-2017-Final-Project'

## Mona's local paths
#BASE = '/Users/mona/OneDrive/repos/Data' #'/home/mmillervedam/Data'
#PROJ = '/Users/mona/OneDrive/repos/final_proj/W266-Fall-2017-Final-Project'#'/home/mmillervedam/ProjectRepo'

In [7]:
## Repo paths
FPATH_EN = BASE + '/Data/test/wiki_en_10K.txt' # first 10000 lines from wiki dump
FPATH_ES = BASE + '/Data/test/wiki_es_10K.txt' # first 10000 lines from wiki dump
EN_ES_DICT = PROJ +'/XlingualEmb/data/dicts/en.es.panlex.all.processed'
EN_IT_DICT  = PROJ +'/XlingualEmb/data/dicts/en.it.panlex.all.processed'
EN_IT_RAW = PROJ + '/XlingualEmb/data/mono/en_it.shuf.10k'

## Large datasets
FULL_EN_ES = "/home/miwamoto/shuffled_files/en_es_shuf.txt"
FULL_EN_IT = "/home/miwamoto/shuffled_files/en_it_shuf.txt"
FULL_EN = BASE + '/Data/en/full.txt'
FULL_ES = BASE + '/Data/es/full.txt'
FULL_IT = BASE + '/Data/it/full.txt'

In [8]:
# directory to save pickled embeddings
SAVE_TO = PROJ + '/Notebooks/trained'

# Load & Preprocess Data
__`ORIGINAL AUTHORS SAY:`__ "Normally, the monolingual word embeddings are trained on billions of words. However, getting that much of monolingual data for a low-resource language is also challenging. That is why we only select the top 5 million sentences (around 100 million words) for each language." - _Section 5.1, Duong et. al._ 

In [9]:
from parsing import Corpus, BilingualVocabulary, batch_generator

### Corpus

In [10]:
# load corpus
en_it_data = Corpus(EN_IT_RAW)

In [7]:
# Corpus Stats
!wc {EN_IT_RAW}

  20000  430887 3746786 /home/mmillervedam/ProjectRepo/XlingualEmb/data/mono/en_it.shuf.10k


In [11]:
# how many batches is 20 epochs? (w/ batch size 48)
430887 // 48 * 20

179520

__`i.e.:`__ 20K sentences (10K in each language) with ~430K tokens
> So this must not be their full data For now, I'm just going to look at the top 20K words and see what happens. In reality we should probably modify the Vocab class so that it explicily collects the top words for each language separately and then concatenates the index.

### Dictionary

In [20]:
# loading english-italian dictionary
pld = pd.read_csv(EN_IT_DICT, sep='\t', names = ['en', 'it'], dtype=str)
en_set = set(pld.en.unique())
it_set = set(pld.it.unique())

In [21]:
# dictionary vocab lengths:
print('EN:', len(en_set))
print('IT:', len(it_set))

EN: 266450
IT: 258641


In [14]:
# Create dictionary for ease of runtime translation
# WARNING this takes a sec to run
bi_dict = pld.groupby(['en'])['it'].unique().to_dict()

In [15]:
# add other direction
# WARNING this takes another sec to run
bi_dict.update(pld.groupby(['it'])['en'].unique().to_dict())

In [22]:
# demo en to it
bi_dict['en_go'][:5]

array(['it_aggirare', 'it_andai', 'it_andara', 'it_andare',
       'it_andare_avanti'], dtype=object)

In [23]:
# demo it to en
bi_dict['it_ciao'][:5]

array(['en_adieu', 'en_bye-bye', 'en_bye', 'en_cheerio', 'en_ciao'], dtype=object)

### Vocabulary

In [24]:
# train multilingual Vocabulary
en_it_vocab = BilingualVocabulary(en_it_data.gen_tokens(), 
                                  languages = ('en','it'), 
                                  size = 30000)

In [18]:
# length of corpus vocabulary
en_it_vocab.size

48579

In [25]:
# transition between words in the bilingual vocab
idx1 = en_it_vocab.lang1_start_idx
idx2 = en_it_vocab.lang2_start_idx
print(idx1 - 1 , en_it_vocab.index[idx1 - 1])
print(idx1, en_it_vocab.index[idx1])
print(idx2 - 1 , en_it_vocab.index[idx2 - 1])
print(idx2, en_it_vocab.index[idx2])

AttributeError: 'BilingualVocabulary' object has no attribute 'lang1_start_idx'

In [37]:
# overlap with dictionary vocabulary
len([w for w in en_it_vocab.types if w in bi_dict])

13447

__`Question for the group`__*Seems low?* Will we limit our vocab to the words in the dict? (MI)

#### Sample of orphaned words

In [24]:
def print_orphans(vocab, bi_dict):
    x = 1
    for w in vocab:
        if w not in bi_dict:
            print(w)
            x += 1
        if x > 20:
            break
            
print_orphans(en_it_vocab.types, bi_dict)

en_manuscripts
en_syro-hittite
en_western-hemisphere
en_migrating
en_conraua
en_aretē
en_tukey
en_trustees
en_240
en_241
en_privileges
en_arab-israeli
en_resembles
en_koyuk
en_schüttler
en_threatened
en_22.3
en_22.2
en_animātiō
en_gamm


### Vocabulary with full corpus

In [29]:
# train multilingual Vocabulary
# NOTE: use FULL_EN_IT if on the instance
en_it_data = Corpus(EN_IT_RAW)

In [52]:
en_it_vocab = Vocabulary(en_it_data.gen_tokens())

In [53]:
# length of corpus vocabulary
en_it_vocab.size

48579

In [54]:
# overlap with dictionary vocabulary
len([w for w in en_it_vocab.types if w in bi_dict])

24176

In [55]:
# sample of orphaned words
print_orphans(en_it_vocab.types, bi_dict )

it_spunti
it_integrali
en_manuscripts
it_giostrando
en_syro-hittite
it_subsessile
it_[[878172]]
it_[[879051]]
it_")
it_raffiguranti
it_promettenti
it_raffigurante
it_margaria
en_western-hemisphere
it_gallizio
en_migrating
en_conraua
it_".
en_aretē
it_macchiano


### CBOW Data Generator
__`CHECK PAPER for HYPERPARAMS!`__: I can't seem to find where they talk about the context window size, embedding size and batch size they use -- it may actually be in the Vulic and Moens paper instead of the Duong one.

__`RLH Update`__: Duong et al. section 6, footnote 4: "Default learning rate of 0.025, negative sampling with 25 samples, subsampling rate of value 1e−4, embedding dimension d = 200, window size cs = 48 and run for 15 epochs"


In [26]:
BATCH_SIZE = 5
WINDOW_SIZE = 4
MAX_EPOCHS = 1 # fail safe

In [59]:
batched_data = batch_generator(en_it_data, 
                               en_it_vocab, 
                               BATCH_SIZE, 
                               WINDOW_SIZE, 
                               MAX_EPOCHS)

Data Format Sanity Check

In [64]:
# STEP 1: recreate sentence & batch generators so they're insynch
token_gen = en_it_data.gen_tokens()
batch_gen = batch_generator(en_it_data, en_it_vocab, BATCH_SIZE, WINDOW_SIZE, MAX_EPOCHS)

In [65]:
# STEP 2: have a look
print("ORIGINAL DATA:")
print([next(token_gen) for _ in range(BATCH_SIZE)])

for context, label in batch_gen:
    print("CONTEXT IDS:", context[:5])
    print("CONTEXT:", [en_it_vocab.to_words(c) for c in context[:5]])
    print("LABEL IDS:", label[:5])
    print("LABELS:", en_it_vocab.to_words(label[:5]))
    break

ORIGINAL DATA:
['it_[[877881]]', 'it_[[879362]]', 'it_in', 'it_un', 'it_remoto']
CONTEXT IDS: [[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 34, 15624, 1584, 309], [0, 0, 0, 20, 15624, 1584, 309, 5533], [0, 0, 20, 34, 1584, 309, 5533, 7]]
CONTEXT: [['<s>', '<s>', '<s>', '<s>', '</s>', '</s>', '</s>', '</s>'], ['<s>', '<s>', '<s>', '<s>', '</s>', '</s>', '</s>', '</s>'], ['<s>', '<s>', '<s>', '<s>', 'it_un', 'it_remoto', 'it_passato', 'it_aveva'], ['<s>', '<s>', '<s>', 'it_in', 'it_remoto', 'it_passato', 'it_aveva', 'it_progettato'], ['<s>', '<s>', 'it_in', 'it_un', 'it_passato', 'it_aveva', 'it_progettato', 'it_,']]
LABEL IDS: [43790, 24849, 20, 34, 15624]
LABELS: ['it_[[877881]]', 'it_[[879362]]', 'it_in', 'it_un', 'it_remoto']


# Fun Validation Words

In [77]:
en_it_vocab.to_ids(['en_the','en_first', 'it_nuovo', 'it_parola'])

[3, 46, 23465, 25963]

In [78]:
bi_dict['en_the']

array(['it_della', 'it_gli', 'it_i', 'it_il', 'it_la', 'it_l\xc3\xa0',
       'it_le', 'it_lo', 'it_ma'], dtype=object)

In [79]:
bi_dict['en_first']

array(['it_anteriore', 'it_anteriormente', 'it_antico', 'it_anzitutto',
       'it_anzi_tutto', 'it_avvio', 'it_dapprima', 'it_davanti',
       'it_di_fronte', 'it_il_primo', 'it_in', 'it_in_cima', 'it_inizio',
       'it_innanzitutto', 'it_innanzi_tutto', 'it_in_primis',
       'it_in_primo_luogo', 'it_la_prima', 'it_per_la_prima_volta',
       'it_per_primo', 'it_precedente', 'it_prima', 'it_prima_di_tutto',
       'it_primariamente', 'it_primario', 'it_prime', 'it_primieramente',
       'it_primiero', 'it_primo', 'it_principale', 'it_principio'], dtype=object)

In [80]:
bi_dict['it_nuovo']

array(['en_fresh', 'en_green', 'en_latter-day', 'en_new', 'en_novel',
       'en_raw', 'en_recent', 'en_renewed', 'en_unexampled', 'en_unused',
       'en_young'], dtype=object)

In [81]:
bi_dict['it_parola']

array(['en_drake', 'en_language', 'en_mot', 'en_parole', 'en_promise',
       'en_-shaped', 'en_speech', 'en_term', 'en_tongue', 'en_verb',
       'en_vocable', 'en_word_of_honor', 'en_word'], dtype=object)

# Base Model - no word sub yet!
__`CODE NOTES:`__ To get this running I had to hard code the context length (set to 2) inside `BuildCoreGraph()` where we generate `self.input_` in line 102. That should really be inferred from the `self.context_` itself but it doesn't seem to like the placeholder dimension (we don't have a span length until runtime). Does tensorflow not have a vectorized average? Something to fix (later). I also had to hard code the number of samples for softmax (I had originally put this as a `tf.placeholder_with_default` thinking we could pass it in to the training function (since its a training parameter) but TF kicked out an error message asking for an integer so for now I'll just give it what it wants. I need to think more about why TF doesn't want this changing from batch to batch. (or if there is another reason it wants an int).

### Fresh Data Generator

In [23]:
BATCH_SIZE = 20
WINDOW_SIZE = 1
MAX_EPOCHS = 15 # fail safe

batched_data = batch_generator(en_it_data, en_it_vocab, BATCH_SIZE, 
                               WINDOW_SIZE, MAX_EPOCHS)

### Initialize the model

In [26]:
from models import BiW2V

EMBEDDING_SIZE = 200

# create model
model = BiW2V(index = en_it_vocab.index, H = EMBEDDING_SIZE)

# intialize TF graphs
model.BuildCoreGraph()
model.BuildTrainingGraph()
model.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Training

__`IMPORTANT!`__ right now the model only works with a window of 1 because the feed dict can't handle context windows of different lengths. We'll either need to figure out how to have a variable length dimension or else add extra padding to the sentences to account for the window size.

In [27]:
# time
start = time.time()

# training parameters
TEST_WORDS = [3, 84, 669, 6646] # en_the, en_first, it_nuovo, it_parole
nBATCHES = 300000 # ~ 14 epochs
DATA_GENERATOR = batched_data

# training call
model.train(nBATCHES, DATA_GENERATOR, TEST_WORDS, learning_rate = 0.15)
tot = (time.time() - start)
print('... {} batches trained in {} seconds'.format(nBATCHES, tot))

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(50709, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(50709, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(50709,) dtype=float32_ref>
... Starting Training
... STEP 0 : Average Loss : 9.23656622569e-05
   [it_,] sim words:  it_piazzati, it_un'iscrizione, it_sessanta, it_verità, it_luo, it_chieri, it_sancita, it_disastrosamente,
   [it_nelle] sim words:  it_solidale, it_percento, it_weiße, it_venerato, it_autori, it_moltissimi, it_organizzando, it_produrli,
   [it_fare] sim words:  it_cambio, it_chomsky, it_catalana, it_meccanismi, it_aiutare, it_agricola, it_incaricato, it_drawing,
   [it_manoscritto] sim words:  it_proietta, it_anticrisi, it_autolinee, it_sv, it_arroscia, it_forze, it_popolato, it_detenuti,
... STEP 30000 : Average Loss : 0.560785405807
... STEP 60000 : Average Loss : 0.353204154999
   [it_,] sim words:  it_piazzati, it_un'iscrizione, it_

__NOTES:__ This is just a context of 1 (ie. window = 3) and there's no bilingual signal. When I ran it w/ the default learning rate there was mad overfitting for `the`'s neighbors but `first` had some much better results (eg. `third` and `only`). It would be interesting to really tune the hyperparamters to see how good we could do (this is essentially monolingual word2vec with two languages)... as a point of comparison for the bilingual versions below.

In [28]:
# take a look at the embeddings
model.context_embeddings

array([[ -4.60977550e-04,   6.87121879e-04,  -6.07755675e-04, ...,
          1.04074273e-03,  -7.94071006e-04,  -4.09811037e-04],
       [  6.01218257e-04,  -1.73640612e-04,   1.50801498e-04, ...,
          9.76910815e-04,   4.34648187e-04,   1.50857595e-04],
       [  4.21131008e-05,   1.43417434e-04,  -4.42015793e-04, ...,
         -2.61730253e-04,   2.63188384e-04,  -3.19738436e-04],
       ..., 
       [ -5.24837349e-04,   7.47645172e-05,  -3.87505628e-04, ...,
          3.38180165e-04,   1.87452009e-04,  -4.23476391e-04],
       [ -2.05034637e-04,  -1.74783956e-04,   2.54430954e-04, ...,
         -1.96897527e-04,  -4.46256425e-04,  -4.10091743e-04],
       [  4.34216403e-04,   9.33892716e-06,  -4.76779445e-04, ...,
          1.30503067e-05,  -3.12790798e-05,  -4.18215437e-04]], dtype=float32)

__`Hmmmm...`__ These don't look normalized to me. Something to return to?

# Model with Random Translation

### Fresh Data

In [27]:
BATCH_SIZE = 48
WINDOW_SIZE = 4
MAX_EPOCHS = 20 # fail safe

batched_data = batch_generator(en_it_data, en_it_vocab, BATCH_SIZE, 
                               WINDOW_SIZE, MAX_EPOCHS)

### Initialize

In [29]:
from models import BiW2V_random

EMBEDDING_SIZE = 200

# create model
model2 = BiW2V_random(bi_dict, vocab = en_it_vocab, H = EMBEDDING_SIZE)

# intialize TF graphs
model2.BuildCoreGraph()
model2.BuildTrainingGraph()
model2.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Train

In [95]:
# training parameters
TEST_WORDS = [3, 46, 23465, 25963] # en_the, en_first, it_nuovo, it_parole
nBATCHES = 100000 # ~ 10 epochs on small en_it dataset
DATA_GENERATOR = batched_data

In [96]:
# training call
start = time.time()
model2.train(nBATCHES, DATA_GENERATOR, TEST_WORDS, learning_rate = 0.15)
tot = (time.time() - start)
print('... {} batches trained in {} seconds'.format(nBATCHES, tot))

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(48579, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(48579, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(48579,) dtype=float32_ref>
... Starting Training
... STEP 0 : Average Loss : 0.000644030761719
   [en_the] sim words:  it_1832, it_paura, en_reduction, en_afroasiatic, it_professionistico, it_primi, en_prosecuted, en_29,689,
   [en_first] sim words:  it_nobiliare, en_child-star, en_protists, it_pensato, en_2:1, it_limitazione, en_theirs, en_countrywide,
   [it_nuovo] sim words:  en_imitation, en_modernize, it_maestri, it_gestito, it_atv, it_contenesse, it_casella, en_observatories,
   [it_parola] sim words:  it_anti, en_fuel, en_babylonians, it_legarsi, en_unspecified, it_reclutato, en_abnormal, it_blockbuster,
... STEP 10000 : Average Loss : 3.72829451993
... STEP 20000 : Average Loss : 3.07080396673
   [en_the] sim words:  en_,, en_of, en_., en_to, 

__`NOTES:`__ Same words look reasonable for 'the' but are subpar for the others. I'd be interesting in training this longer and experimenting with the learning rate. [UPDATE: alpha = 0.55 seemed to perform a little better but mixed results (at least in terms of the example words) with 0.85 and 0.65... going to wait for more data before spending more time on this point.]

In [70]:
# saving final embeddings in case we want to do more stuff later
filename = SAVE_TO + '../embeddings/en_it_rand_600K_cw1_V_dec15.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model2.context_embeddings, f, pickle.HIGHEST_PROTOCOL)

filename = SAVE_TO + '../embeddings/en_it_rand_600K_cw1_U_dec15.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model2.word_embeddings, f, pickle.HIGHEST_PROTOCOL)

In [73]:
# confirm:
filename = SAVE_TO + '/en_it_rand_600K_cw1_U_dec15.pkl'
with open(filename, 'rb') as f:
    C_embedding = pickle.load(f)
    
C_embedding

array([[ -1.42820040e-02,  -1.08593737e-03,  -9.77827888e-03, ...,
         -7.43078999e-03,  -9.79084056e-04,  -7.75979646e-03],
       [  6.15398725e-03,   3.99457384e-03,  -7.21636403e-04, ...,
         -4.45156882e-04,   4.76947054e-03,   4.01509507e-03],
       [  1.42526999e-03,  -2.62570567e-03,   6.83171733e-04, ...,
         -2.14850740e-03,  -6.21526386e-04,   8.00127018e-05],
       ..., 
       [ -4.83615768e-05,  -7.35577720e-04,   2.69850646e-03, ...,
         -1.72964076e-03,   2.55509047e-03,   6.92204339e-04],
       [  3.46941641e-03,   5.76911087e-04,   7.60798051e-04, ...,
          3.49387084e-03,   3.47503019e-03,  -1.87479728e-03],
       [ -7.38707022e-04,  -1.68911624e-03,  -2.75655207e-03, ...,
         -2.23345775e-03,  -2.73358473e-03,   1.52106478e-03]], dtype=float32)

# Random Translation with Larger Context

__`WARNING:`__ the code below was run on an old version of the model class

### Fresh Data

In [140]:
BATCH_SIZE = 20
WINDOW_SIZE = 8
MAX_EPOCHS = 30 # fail safe

batched_data = batch_generator(en_it_data, en_it_vocab, BATCH_SIZE, 
                               WINDOW_SIZE, MAX_EPOCHS)

### Initialize Model

In [141]:
from models import BiW2V_random

EMBEDDING_SIZE = 200

# create model
model3 = BiW2V_random(('en', 'it'), bi_dict, en_it_vocab.to_ids,
                      index = en_it_vocab.index, 
                      H = EMBEDDING_SIZE)

# intialize TF graphs
model3.BuildCoreGraph()
model3.BuildTrainingGraph()
model3.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Train

In [142]:
# parameters
nBATCHES = 600000 # ~ 25 epochs
DATA_GENERATOR = batched_data
TEST_WORDS = [3, 84, 669, 6646]

# training call
model3.train(nBATCHES, DATA_GENERATOR, TEST_WORDS, learning_rate = 0.15)

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(48579, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(48579, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(48579,) dtype=float32_ref>
... Starting Training
... STEP 0 : Average Loss : 0.000122350947062
   [en_the] sim words:  en_diakonoff, it_abraham, it_iuta, en_coups, en_griffith, en_alter, en_upward, en_1770s,
   [en_first] sim words:  it_firmato, en_boil, it_fiocina, it_vga, it_[[876681]], en_featural, it_risparmio, en_run,
   [it_nuovo] sim words:  it_sabana, it_dell’antimateria, en_stabbing, en_probes, it_hetman, it_rof, it_roseto, en_restarted,
   [it_parola] sim words:  it_d'artificio, it_rimanesse, en_infantile, it_branca, it_permesso, en_materialism, en_reunited, en_non-syndromal,
... STEP 60000 : Average Loss : 3.43592488782
... STEP 120000 : Average Loss : 2.32015497759
   [en_the] sim words:  en_counteracts, en_antiquated, en_tropics, it_1829,

__NOTES:__ Interesting, the larger context seems to hurt the training performance. Is this because of the extra padding? Or will a smart adjustment of the learning rate redeem this?

# Vizualization

In [106]:
wrds = "en_the en_a en_this en_'s en_an en_their en_its en_these en_his \
       en_first en_on en_in en_for en_to en_with en_are en_. en_all \
       it_nuovo it_di it_un it_, <s> it_i it_con it_è it_più it_parola \
       en_censorship en_minima it_profesional it_michail en_الأول \
       it_ritengono en_prevention it_mckenna en_third".split()

In [107]:
for w in ["en_the", "en_first", "it_nuovo", "it_parole"]:
    wrds += list(bi_dict[w])

In [108]:
wordset = set(en_it_vocab.to_ids(wrds))

In [110]:
model2.plot_embeddings_in_2D(wordset)

NameError: global name 'TSNE' is not defined

# Evaluation

In [30]:
source_lang = "en"
target_lang="it"
GTT_BASE = '/home/rhopper/W266-Fall-2017-Final-Project/BaselineModels/data/ground_truth_translations/' #'/home/mmillervedam/ProjectRepo/BaselineModels/data/ground_truth_translations/'
GTT_PATH = GTT_BASE + "%s-%s-clean.txt" % (source_lang, target_lang)
gtt = pd.read_csv(GTT_PATH, names = [source_lang, target_lang], sep=" ", header=None)

In [39]:
model2.evaluate(ground_truth_translations=gtt, sample=[3, 4, 5])

... Model Initialized
sim shape (3, 48579)
vocab size 48579
word en_the
half of vocab 24291


IndexError: index 24291 is out of bounds for axis 0 with size 3